In [2]:
# Python v3.10.4
import re
import numpy as np
import networkx as nx #v2.8.2
import random as rnd
import itertools
from networkx.utils.decorators import py_random_state
import bisect
import time

--------------------------------------
<h3>Creation graph from dataset</h3>
<h5>ID odd -> film <br>
    ID even -> actor</h5>

In [3]:
file=open('imdb-actors-actresses-movies.tsv','r')
EOF=False
id_actor=0
id_film=1
count_actor=0
count_film=1
actor_to_key={}
key_to_actor={}
film_to_key={}
key_to_film={}
G=nx.Graph()
while not EOF:
    line=file.readline().split('\n')
    record=line[0].split('\t')
    if  len(record)<2:
        print(record)
        EOF=True
    else:
        film=record[1]
        year=re.search("\s\([0-9]{4}\)",film)
        actor=record[0]
        if year:
            year=np.int16(year.group(0).replace(" (",'').replace(')',''))
        else:
            year=re.search("\s\([0-9]{4}\/",film)
            if year:
                year=np.int16(year.group(0).replace(" (",'').replace('/',''))
            else:
                year=3000 #fake year
        if actor not in actor_to_key:
            id_actor=count_actor
            count_actor=count_actor+2
            actor_to_key[actor]=id_actor
            key_to_actor[id_actor]=actor
        else:
            id_actor=actor_to_key[actor]
        if film not in film_to_key:
            id_film=count_film
            count_film=count_film+2
            film_to_key[film]=id_film
            key_to_film[id_film]=(film,year)
        else:
            id_film=film_to_key[film]
        G.add_edge(id_actor,id_film,year=year)
del film_to_key
del actor_to_key
del id_actor
del id_film
del count_actor
del count_film

['']


--------------------------------------
<h3>Which is the movie with the largest number of actors, considering only the movies up to year x?</h3>

In [ ]:
x = [1930,1940,1950,1960,1970,1980,1990,2000,2010,2020]
up_to_year=x[rnd.randint(0,len(x)-1)]
time_start=time.time()
best_result={'count':0,'movies':[]}
for film,name_year in key_to_film.items():
    if name_year[1]<=up_to_year:
        tot_actors=len(G[film])
        if(tot_actors>=best_result['count']):
            if tot_actors>best_result['count']:
                best_result['movies']=[]
                best_result['count']=tot_actors
            best_result['movies'].append(film)
print('up to year -> '+str(up_to_year))
print('Number of actors -> '+str(best_result['count']))
for movie in best_result['movies']:
    print('Film -> '+key_to_film[movie][0])

----------------------------------------------------------
<h3>Considering only the movies up to year x with x in {1930,1940,1950,1960,1970,1980,1990,2000,2010,2020} and restricting to the largest connected component of the graph.<br> 
Compute exactly the diameter of G</h3>


In [9]:
@py_random_state(1)
def two_sweep(Graph, seed):
    # select a random source node
    rnd_node = seed.choice(list(Graph))
    # get the distances to the other nodes
    source= list(nx.single_source_shortest_path_length(Graph, rnd_node))[-1] #____
    # take a node that is (one of) the farthest nodes from the source
    distances_b = list(nx.single_source_shortest_path_length(Graph, source).items())
    index_start_node=bisect.bisect_left(distances_b,int(distances_b[-1][1]/2),key=lambda k:k[1]) #start all nodes at distance/2
    index_end_node=bisect.bisect_right(distances_b,int(distances_b[-1][1]/2),key=lambda k:k[1]) #end _ 
    return max(distances_b[index_start_node::index_end_node],key=lambda k:len(H_small[k[0]])),distances_b[-1][1] #highest degree

In [12]:
def iFUB(G,node_start):
    list_fringe=itertools.groupby(reversed(nx.single_source_shortest_path_length(G,node_start).items()),key=lambda k:k[1])
    group=next(list_fringe)
    i=group[0]
    lb=i
    ub=2*i
    max_ecc=0
    count_bfs=1
    while ub>lb:
        for element in group[1]:
            ecc=nx.eccentricity(G,element[0])
            count_bfs+=1
            if ecc>(2*(i-1)):
                return ecc
            if ecc>max_ecc:
                max_ecc=ecc
        lb=max(lb,max_ecc)
        ub=2*(i-1)
        i=i-1
        group=next(list_fringe)
    print('Nr bfs -> '+str(count_bfs))
    return lb

In [13]:
x = [1930,1940,1950,1960,1970,1980,1990,2000,2010,2020]
year=rnd.choice(x)
time_start=time.time()
list_nodes=set()
for u,v in G.edges():
    if G[u][v]['year'] <= year:
        list_nodes.add(u)
        list_nodes.add(v)
largest_cc=max(nx.connected_components(G.subgraph(list_nodes)),key=len)
H_small=G.subgraph(largest_cc)
del(largest_cc)
del(list_nodes)
print('Only films up to '+str(year))
node_start,lb=two_sweep(H_small,None)
print(lb)
diameter=iFUB(H_small,node_start[0])
time_end=time.time()
print('Diameter -> '+str(diameter))
print('Time -> '+str(time_end-time_start))

Only films up to 1930
32
Nr bfs -> 34
Diameter -> 32
Time -> 51.34049940109253


--------------------------------------------------
<h3>Which is the movie with the largest number of popular actors, i.e. such that the sum of the number of movies its actors participated in is maximum? For example movie M contains actors A and B. A participated in 5 movies, B in 3. The score of M is 8.</h3>

In [ ]:
largest_movie={'count':0,'movies':[]}  
for film,_ in itertools.chain(key_to_film.items()):
    count=sum(G.degree(act) for _,act in itertools.chain(G.edges(film)))
    if count>=largest_movie['count']:
        if count>largest_movie['count']:
            largest_movie['count']=count
            largest_movie['movies']=[]
        largest_movie['movies'].append(film)
print(largest_movie)
for i in largest_movie['movies']:
    print(key_to_film[i])

-------------------------------------------
<h3>Build also the actor graph, whose nodes are only actors and two actors are connected if they did a movie together.<br>
Which is the pair of actors who collaborated the most among themselves?</h3>


In [ ]:
def create_graph_actors_partially_ordered(): 
    collaborations={}
    best_collaborators=[0,[]]
    G_actor=nx.Graph()
    count=0
    for actor_1 in key_to_actor:
        count+=1
        list_films=itertools.chain(G[actor_1])
        for film in list_films:
            list_actor_for_film=itertools.chain(G[film])
            for actor_2 in list_actor_for_film:
                if actor_2>actor_1: # avoid duplication
                    if actor_2 in collaborations:
                        collaborations[actor_2]+=1
                        if collaborations[actor_2]>=best_collaborators[0]:
                            if collaborations[actor_2]>best_collaborators[0]:
                                best_collaborators[0]=collaborations[actor_2]
                                best_collaborators[1]=[]
                            best_collaborators[1].append(actor_2)
                    else:
                        collaborations[actor_2]=1
        for element in itertools.chain(best_collaborators[1]): # pick max items and add them as last
            G_actor.add_edge(actor_1,element,weight=best_collaborators[0])
        for element in (collaborations.items()):
            if element[0] not in best_collaborators[1]:
                G_actor.add_edge(actor_1,element[0],weight=element[1])
        collaborations.clear()
        best_collaborators=[0,[]]
        if count%300_000==0:
            print(str(count/len(key_to_actor)*100)+' %')
    return G_actor

In [ ]:
# 6 min
G_actor=create_graph_actors_partially_ordered()

In [ ]:
# 4:10 min
best_result={'count':0,'pairs':[]} 
count=0
for actor in itertools.chain(G_actor):
    count+=1
    for actor_2 in itertools.chain(((G_actor[actor]))):
        if actor_2>actor:
            colls=G_actor[actor][actor_2]['weight']
            if colls>=best_result['count']:
                if colls>best_result['count']:
                    best_result['count']=colls
                    best_result['pairs']=[]
                best_result['pairs'].append((actor,actor_2))
            else:
                break
    if count%300_000==0:
        print(str(count/len(key_to_actor)*100)+' %')
best_result